# 🧠 Linux User Behavior & Session Analysis
This notebook helps incident responders analyze user login behavior, shell history, and privilege escalation events on a Linux system using forensic logs. Please note, you WILL need to modify paths to point to mounted evidence and make sure dependencies are met.

In [ ]:
# 📦 Setup & Imports
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display

sns.set(style="darkgrid")

In [ ]:
# 📁 File Presence Check
log_files = {
    "audit": "/var/log/audit/audit.log",
    "bash_history": os.path.expanduser("~/.bash_history"),
    "zsh_history": os.path.expanduser("~/.zsh_history"),
    "wtmp": "/var/log/wtmp",
    "btmp": "/var/log/btmp",
    "auth": "/var/log/auth.log"  # or /var/log/secure on RHEL
}

for name, path in log_files.items():
    print(f"{name}: {'FOUND' if os.path.exists(path) else 'MISSING'}")

In [ ]:
# 🕓 Login Timeline from wtmp
def parse_last_output(file_path):
    from subprocess import check_output
    output = check_output(["last", "-F", "-f", file_path], text=True)
    lines = [line for line in output.splitlines() if line and not line.startswith("wtmp begins")]
    records = []
    for line in lines:
        match = re.match(r"(\S+)\s+(\S+)\s+(\S+)\s+(.+)", line)
        if match:
            user, tty, ip, timestamp = match.groups()
            records.append({
                "user": user,
                "tty": tty,
                "ip": ip,
                "timestamp": timestamp.strip()
            })
    return pd.DataFrame(records)

wtmp_df = parse_last_output(log_files["wtmp"])
wtmp_df["timestamp"] = pd.to_datetime(wtmp_df["timestamp"], errors="coerce")
display(wtmp_df.head())

In [ ]:
# 📈 Plot login timeline
plt.figure(figsize=(12, 4))
sns.histplot(wtmp_df["timestamp"], bins=50, kde=False)
plt.title("User Login Timeline")
plt.xlabel("Time")
plt.ylabel("Login Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 🌍 Source IP Frequency
ip_counts = wtmp_df["ip"].value_counts().reset_index()
ip_counts.columns = ["IP Address", "Login Count"]

plt.figure(figsize=(10, 5))
sns.barplot(data=ip_counts.head(10), x="IP Address", y="Login Count")
plt.title("Top Source IPs for Login")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 🔐 SUDO / SU Usage from auth.log
def parse_auth_log(path):
    sudo_events = []
    su_events = []
    with open(path) as f:
        for line in f:
            if "sudo:" in line:
                sudo_events.append(line.strip())
            elif "session opened for user" in line and "su" in line:
                su_events.append(line.strip())
    return sudo_events, su_events

sudo_logs, su_logs = parse_auth_log(log_files["auth"])
print(f"SUDO events: {len(sudo_logs)}")
print(f"SU events: {len(su_logs)}")

In [ ]:
# 🧑‍💻 Shell History Analysis
def parse_shell_history(path):
    if not os.path.exists(path):
        return pd.DataFrame()
    with open(path) as f:
        lines = f.readlines()
    has_timestamps = any(line.startswith("#") for line in lines)
    records = []
    timestamp = None
    for line in lines:
        line = line.strip()
        if line.startswith("#"):
            try:
                timestamp = datetime.fromtimestamp(int(line[1:]))
            except:
                timestamp = None
        elif line:
            records.append({
                "timestamp": timestamp,
                "command": line
            })
    return pd.DataFrame(records)

bash_df = parse_shell_history(log_files["bash_history"])
zsh_df = parse_shell_history(log_files["zsh_history"])
history_df = pd.concat([bash_df, zsh_df]).dropna(subset=["command"])
display(history_df.head())

In [ ]:
# 📊 Command Frequency
cmd_counts = history_df["command"].str.split().str[0].value_counts().head(10)
cmd_counts.plot(kind="bar", figsize=(10, 4), title="Top Commands Used")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 🕒 Optional: Command Usage by Hour
if not history_df.empty:
    history_df["hour"] = history_df["timestamp"].dt.hour
    sns.histplot(history_df["hour"], bins=24)
    plt.title("Command Usage by Hour")
    plt.xlabel("Hour of Day")
    plt.ylabel("Command Count")
    plt.tight_layout()
    plt.show()